In [13]:
import sqlite3
import pandas as pd

In [14]:
df = pd.read_csv("../data/insurance_sql_ready.csv")

In [15]:
#SQLite database
conn = sqlite3.connect("../data/insurance.db")

In [17]:
# Write to a SQL table
df.to_sql("insurance", conn, if_exists="replace", index=False)

1337

In [19]:
# Confirm row count
pd.read_sql("SELECT COUNT(*) AS row_count FROM insurance;", conn)

,row_count
0,1337


In [22]:
query = """
SELECT
    smoker,
    COUNT(*) AS n,
    ROUND(AVG(charges), 2) AS avg_charges
FROM insurance
GROUP BY smoker
ORDER BY avg_charges DESC;
"""

pd.read_sql(query, conn)

,smoker,n,avg_charges
0,yes,274,32050.23
1,no,1063,8440.66


In [28]:
query = """
SELECT
    CASE
        WHEN age BETWEEN 18 AND 29 THEN '18-29'
        WHEN age BETWEEN 30 AND 39 THEN '30-39'
        WHEN age BETWEEN 40 AND 49 THEN '40-49'
        WHEN age BETWEEN 50 AND 59 THEN '50-59'
        ELSE '60+'
    END AS age_group,
    COUNT(*) AS n,
    ROUND(AVG(charges), 2) AS avg_charges
FROM insurance
GROUP BY age_group
ORDER BY age_group;
"""
pd.read_sql(query, conn)

,age_group,n,avg_charges
0,18-29,416,9200.62
1,30-39,257,11738.78
2,40-49,279,14399.20
3,50-59,271,16495.23
4,60+,114,21248.02


In [26]:
query = """
SELECT
    region,
    COUNT(*) AS n,
    ROUND(AVG(charges), 2) AS avg_charges
FROM insurance
GROUP BY region
ORDER BY avg_charges DESC;
"""
pd.read_sql(query, conn)

,region,n,avg_charges
0,southeast,364,14735.41
1,northeast,324,13406.38
2,northwest,324,12450.84
3,southwest,325,12346.94


In [30]:
query = """
SELECT
    smoker,
    COUNT(*) ASn,
    ROUND(AVG(bmi), 2) AS avg_bmi,
    ROUND(AVG(bmi_smoker), 2) AS avg_bmi_smoker,
    ROUND(AVG(charges), 2) AS avg_charges
FROM insurance
GROUP BY smoker;
"""
pd.read_sql(query, conn)

,smoker,ASn,avg_bmi,avg_bmi_smoker,avg_charges
0,no,1063,30.65,0.00,8440.66
1,yes,274,30.71,30.71,32050.23


In [31]:
query = """
SELECT
    age, sex, bmi, children, smoker, region, bmi_smoker,
    ROUND(charges, 2) AS charges
FROM insurance
ORDER BY charges DESC
LIMIT 10;
"""
pd.read_sql(query, conn)

,age,sex,bmi,children,smoker,region,bmi_smoker,charges
0,54,female,47.410,0,yes,southeast,47.410,63770.43
1,45,male,30.360,0,yes,southeast,30.360,62592.87
2,52,male,34.485,3,yes,northwest,34.485,60021.40
3,31,female,38.095,1,yes,northeast,38.095,58571.07
4,33,female,35.530,0,yes,northwest,35.530,55135.40
5,60,male,32.800,0,yes,southwest,32.800,52590.83
6,28,male,36.400,1,yes,southwest,36.400,51194.56
7,64,male,36.960,2,yes,southeast,36.960,49577.66
8,59,male,41.140,1,yes,southeast,41.140,48970.25
9,44,female,38.060,0,yes,southeast,38.060,48885.14


In [20]:
query = """
SELECT
  CASE
    WHEN smoker = 'yes' AND bmi >= 30 THEN 'High risk: smoker & obese'
    WHEN smoker = 'yes' THEN 'High risk: smoker'
    WHEN bmi >= 30 THEN 'Moderate risk: obese'
    ELSE 'Lower risk'
  END AS risk_segment,
  COUNT(*) AS n,
  ROUND(AVG(charges), 2) AS avg_charges
FROM insurance
GROUP BY risk_segment
ORDER BY avg_charges DESC;
"""
pd.read_sql(query, conn)


,risk_segment,n,avg_charges
0,High risk: smoker & obese,145,41557.99
1,High risk: smoker,129,21363.22
2,Moderate risk: obese,561,8855.53
3,Lower risk,502,7977.03
